# Calculation 03 - Point defect energies
The following notebook will calculate the point defect energies of Nickel-based EAM and MEAM potentials. The first cell needs updating depending on the potential file you are using and the compound you want to run the calculation for.\
At the end of the notebook, there is a "fast run" which will allow you to run this notebook for all potentials and all phases in one go.

## 📌 Specify filenames and element related data
For any potential, please specify the filename of the potential file as follows:\
`potentials/filename.eam`\
`potentials/filename.eam.fs`\
`potentials/filename.eam.alloy`\
`potentials/filename.meam`

Then uncomment the respective lines, depending on which compound you want to run the calculation for.

**Note:** to call Non-LAMMPS variables within a LAMMPS command, please use an f-string. To call LAMMPS variables, instead of using an f-string, just use curly braces with a leading dollar sign - like this: `${x}`.

In [ ]:
potentialpath = "potentials/Ni1986.eam"

#pointdefect = "V_Ni"
#pointdefect = "Al_Ni"
#pointdefect = "V_Al"
#pointdefect = "Ni_Al"

##### insert whichelements, crystalstructure, latticeconstant and outputfile here #####


'''
#fcc-Ni
compound = "Ni"
whichelements = "Ni"
crystalstructure = "fcc"
latticeconstant = 3.52

#B2-NiAl
compound = "NiAl"
whichelements = "Ni Al"
crystalstructure = "bcc"
latticeconstant = 2.86

#L12-Ni3Al
compound = "Ni3Al"
whichelements = "Ni Al"
crystalstructure = "fcc"
latticeconstant = 3.56
'''

outputfile = "data/03_point-defects_"+compound+".dat"

## ⬇️ Import PyLammps module

In [ ]:
from lammps import PyLammps
lmp = PyLammps()

lmp.clear()

## ⬇️ Add parameters for MEAM potentials
There is no need to change the following cell. It defines two parameters that are necessary for reading in MEAM potentials.\
**Please do note that the libraryelements variable has to contain the element symbols in the same order as they appear in the library file!**

In [ ]:
if "meam" in potentialpath:
    libraryfile = str(potentialpath.replace('.meam', '.library.meam'))
    
    if "NiAlCo2" in potentialpath:
        libraryelements = "Ni Al Co"
    elif "NiAl2" in potentialpath:
        libraryelements = "Al Ni"
    else:
        libraryelements = "Ni"

## ⬇️ Initialize simulation
`units` sets the unit system for this simulation. "metal" units are Angstroms for distance, eV for energy, etc.\
`dimension` is self-explanatory and can either be 2 or 3.\
`boundary` sets the boundary conditions in x-, y- and z-direction. Here, they are periodic in all three dimensions.\
`atom_style` specifies additional attributes depending on the style - "atomic" doesn't have any.

In [ ]:
lmp.units('metal')
lmp.dimension('3')
lmp.boundary('p p p')
lmp.atom_style('atomic')

## ⬇️ Create atoms
`lattice` specifies the predefined or customized lattice and the respective lattice constant.\
`region` specifies the simulation cell as a geometric shape, like a sphere, cylinder or block. The default coordinate unit is a "lattice parameter", meaning each coordinate is an integer factor times the lattice parameter specified in the `lattice` command. "cell" is the user-assigned ID for this specific region, but could also be called "1", "a" or anything else. For periodic boundary conditions, the cell dimensions should be multiples of the lattice constant to prevent unwanted overlaps (as explained in the documentation for the `create_atoms` command).\
`create_box` actually creates the simulation cell as defined in the `region` command. The integer defines the number of element types that will be used, the string defines the region in which the cell will be created.\
`create_atoms` adds atoms to the lattice within a specific region ("box" fills the entire simulation cell defined in the `region` command with atoms), so this has to happen after defining and creating the simulation box. The integer defines the number of element types used. "basis" allows to assign coordinates to atom types: the first integer is the coordinate (they can be found in the documentation for the `lattice` command), the second one is the element index in the same order as it will be specified when reading in the potentials (Ni = 1, Al = 2).
`replicate` would allow to change the size of the simulation, and in this position is equivalent to changing the coordinates in the `region` command.
`mass` does not need to specified for any EAM or MEAM potential since it is used from the potential file itself.

In [ ]:
lmp.lattice(f'{crystalstructure} {latticeconstant}')
lmp.region('cell block -3 3 -3 3 -3 3')

if compound == "Ni":
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')
elif compound == "NiAl":
    lmp.create_box('2 cell')
    lmp.create_atoms('2 box basis 1 1 basis 2 2')
elif compound == "Ni3Al":
    lmp.create_box('2 cell')
    lmp.create_atoms('2 box basis 1 2 basis 2 1 basis 3 1 basis 4 1')

## ⬇️ Read in potential file
`pair_style` specifies what kind of interatomic potential will be used.\
`pair_coeff` specifies the potential file to be read in. The input parameters change depending on the potential style. that the pair potential coefficients are stored in. The first two integers define the force field coefficients between atom pairs. The asterisks include all atom types.\
`neighbor` and `neigh_modify` would set parameters for the neighbor lists - the default parameters when the commands are not used can be found in the LAMMPS documentation.

In [ ]:
if "meam" in potentialpath:
    lmp.pair_style('meam')
    lmp.pair_coeff(f'* * {libraryfile} {libraryelements} {potentialpath} {whichelements}')
elif "eam.alloy" in potentialpath:
    lmp.pair_style('eam/alloy')
    lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
elif "eam.fs" in potentialpath:
    lmp.pair_style('eam/fs')
    lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
else:
    lmp.pair_style('eam')
    lmp.pair_coeff(f'* * {potentialpath}')

## ⬇️ Define computes
`compute`s define "formulas" that can later on be computed at each timestep of e.g. a thermodynamic run, minimization or dump output.\
The first one allows to store the potential energy for each atom during any "run".\
The second one uses the first one, summing up the potential energies of all atoms in the system.\
The third one allows to calculate the centro-symmetry parameter.

In [ ]:
lmp.compute('Epot all pe/atom')
lmp.compute('Epotall all reduce sum c_Epot')
lmp.compute('Csym all centro/atom fcc')

## ⬇️ Define thermo settings
`min_style` would set which minimization algorithm should be used for the minimization. The default value is `cg` - conjugate gradient.\
`thermo` sets the timestep interval at which computes shall be performed and printed during the run.\
`thermo_style` allows to specify what information shall be printed at each of those timesteps - in this case, it is customized to calculate and print the timestep, the length of the simulation box in x, y and z direction, and the total potential energy of the system.

In [ ]:
lmp.thermo(10)
lmp.thermo_style('custom step lx ly lz c_Epotall')

## ⬇️ Define dump file
`dump` will print the defined properties in a dump file at the desired timestep interval. With these files, the simulation can be visualized using software like OVITO.

In [ ]:
lmp.dump(f'dump all cfg 1000 dump/03_{compound}_{pointdefect}.*.cfg mass type xs ys zs c_Csym c_Epot')
lmp.dump_modify(f'dump element {whichelements}')

## ⬇️ Run minimization I
`minimize`s the energy of the simulation cell, printing out the desired information (as specified in `thermo_style`) at every `thermo(10)`th timestep.

In [ ]:
lmp.minimize('1e-10 1e-10 10000 100000')

## ⬇️ Define variable formulas
This section defines some `variable`s by assigning a variable to a formula that is not yet evaluated.\
The first one is the total number of atoms in the simulation cell N, the second one the total energy of the simulation cell E. 

In [ ]:
lmp.variable('N equal count(all)')
lmp.variable('E equal c_Epotall')

## ⬇️ Calculate properties I
Here, the formulas defined above are calculated with the current values of the formula variables and stored in new variables.

In [ ]:
lmp.variable('Ni equal ${N}')
lmp.variable('Ei equal ${E}')

## ⬇️ Generate vacancy
A variable calculating the size of the region within which a particle will be removed from is defined. This is done by calculating the radius of one Ni atom. This is pure geometry: the diagonal of the cube face of the fcc unit cell (Pythagoras: r<sup>2</sup>+r<sup>2</sup>) is four atomic radii long.\
`region` defines the region in which atoms will be removed - hence the calculation of the atom size of one atom.\
`delete_atoms` actually deletes the atom(s) in the defined region.

In [ ]:
lmp.variable(f'distance equal {latticeconstant}/2')

#define region where we can find a nickel atom
if pointdefect == "V_Ni" or pointdefect == "Al_Ni":
    
    lmp.variable('radius equal sqrt(2)*3.52/4')
        
    if compound == "Ni":        
        lmp.region('vacancy sphere 0 0 0 ${radius} units box')

    elif compound == "NiAl":
        lmp.region('vacancy sphere 0 0 0 ${radius} units box')

    elif compound == "Ni3Al":
        lmp.region('vacancy sphere ${distance} ${distance} 0 ${radius} units box')

#define region where we can find an aluminum atom
elif pointdefect == "V_Al" or pointdefect == "Ni_Al":
    
    lmp.variable('radius equal sqrt(2)*4.05/4')
    
    if compound == "NiAl":
        lmp.region('vacancy sphere ${distance} ${distance} ${distance} ${radius} units box')

    elif compound == "Ni3Al":
        lmp.region('vacancy sphere 0 0 0 ${radius} units box')

#actually delete the atom
lmp.delete_atoms('region vacancy compress yes')

#put an aluminum atom in the nickel vacancy
if pointdefect == "Al_Ni":
        
    if compound == "NiAl":
        lmp.create_atoms('2 single 0 0 0 units box')

    elif compound == "Ni3Al":
        lmp.create_atoms('2 single ${distance} ${distance} 0 units box')

#put a nickel atom in the aluminum vacancy
elif pointdefect == "Ni_Al":

    if compound == "NiAl":
        lmp.create_atoms('1 single ${distance} ${distance} ${distance} units box')

    elif compound == "Ni3Al":
        lmp.create_atoms('1 single 0 0 0 units box')

## ⬇️ Run minimization II
`minimize`s the energy of the simulation cell, printing out the desired information (as specified in `thermo_style`) at every `thermo(10)`th timestep.

In [ ]:
lmp.minimize('1e-10 1e-10 10000 100000')

## ⬇️ Calculate properties II
Again, the formulas defined above are calculated with the current values of the formula variables and stored in new variables.\
A third variable, calculating the vacancy formation energy, is defined and will be calculated in the `print` command with the current variable values.

In [ ]:
lmp.variable('Nf equal ${N}')
lmp.variable('Ef equal ${E}')

lmp.variable(f'E_{pointdefect}'+' equal ${Ef}-${Ei}')

## ⬇️ Undump
Close dump file.

In [ ]:
lmp.undump('dump')

## ⬇️ Print desired output
This section `print`s these values to the output file, for which you chose a filename in the beginning. It prints the filename, the cohesive energy and the lattice parameter of the chosen potential.

In [ ]:
potentialname = potentialpath.replace('potentials/', '')

lmp.print(f'"{potentialname}, {pointdefect}'+', ${Ei}, ${Ef}, ${E_'+f'{pointdefect}'+'}" append '+f'{outputfile} screen no')

🏁**ALL DONE!**🏁

In [ ]:
def vacancyformation(compound, whichelements, crystalstructure, latticeconstant, pointdefect, potentialpath):
    lmp.clear()

    if "meam" in potentialpath:
        librarypath = str(potentialpath.replace('.meam', '.library.meam'))

    if "NiAlCo2" in potentialpath:
        libraryelements = "Ni Al Co"
    elif "NiAl2" in potentialpath:
        libraryelements = "Al Ni"
    else:
        libraryelements = "Ni"

    lmp.units('metal')
    lmp.dimension('3')
    lmp.boundary('p p p')
    lmp.atom_style('atomic')
    
    lmp.lattice(f'{crystalstructure} {latticeconstant}')
    lmp.region('cell block -3 3 -3 3 -3 3')

    if compound == "Ni":
        lmp.create_box('1 cell')
        lmp.create_atoms('1 box')
    elif compound == "NiAl":
            lmp.create_box('2 cell')
            lmp.create_atoms('2 box basis 1 1 basis 2 2')
    elif compound == "Ni3Al":
            lmp.create_box('2 cell')
            lmp.create_atoms('2 box basis 1 2 basis 2 1 basis 3 1 basis 4 1')
    
    if "meam" in potentialpath:
        lmp.pair_style('meam')
        lmp.pair_coeff(f'* * {librarypath} {libraryelements} {potentialpath} {whichelements}')
    elif "eam.alloy" in potentialpath:
        lmp.pair_style('eam/alloy')
        lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
    elif "eam.fs" in potentialpath:
        lmp.pair_style('eam/fs')
        lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
    else:
        lmp.pair_style('eam')
        lmp.pair_coeff(f'* * {potentialpath}')

    lmp.compute('Epot all pe/atom')
    lmp.compute('Epotall all reduce sum c_Epot')

    lmp.reset_timestep(0)

    lmp.thermo(10)
    lmp.thermo_style('custom step lx ly lz c_Epotall')

    lmp.minimize('1e-10 1e-10 10000 100000')
    
    lmp.variable('N equal count(all)')
    lmp.variable('E equal c_Epotall')
    
    lmp.variable('Ni equal ${N}')
    lmp.variable('Ei equal ${E}')
    
    lmp.variable(f'distance equal {latticeconstant}/2')

    if pointdefect == "V_Ni" or pointdefect == "Al_Ni":
        
        lmp.variable('radius equal sqrt(2)*3.52/4')

        if compound == "Ni":
            lmp.region('vacancy sphere 0 0 0 ${radius} units box')

        elif compound == "NiAl":
            lmp.region('vacancy sphere 0 0 0 ${radius} units box')

        elif compound == "Ni3Al":
            lmp.region('vacancy sphere ${distance} ${distance} 0 ${radius} units box')

    elif pointdefect == "V_Al" or pointdefect == "Ni_Al":
        
        lmp.variable('radius equal sqrt(2)*4.05/4')

        if compound == "NiAl":
            lmp.region('vacancy sphere ${distance} ${distance} ${distance} ${radius} units box')

        elif compound == "Ni3Al":
            lmp.region('vacancy sphere 0 0 0 ${radius} units box')

    lmp.delete_atoms('region vacancy compress yes')

    if pointdefect == "Al_Ni":

        if compound == "NiAl":
            lmp.create_atoms('2 single 0 0 0 units box')

        elif compound == "Ni3Al":
            lmp.create_atoms('2 single ${distance} ${distance} 0 units box')

    elif pointdefect == "Ni_Al":

        if compound == "NiAl":
            lmp.create_atoms('1 single ${distance} ${distance} ${distance} units box')

        elif compound == "Ni3Al":
            lmp.create_atoms('1 single 0 0 0 units box')
    
    lmp.minimize('1e-10 1e-10 10000 100000')
    
    lmp.variable('Nf equal ${N}')
    lmp.variable('Ef equal ${E}')  
    
    if compound == "Ni":
        lmp.variable(f'E_{pointdefect}'+' equal ${Ef}-(${Nf}/${Ni})*${Ei}')
            
    elif compound == "NiAl" or compound == "Ni3Al":
        lmp.variable(f'E_{pointdefect}'+' equal ${Ef}-${Ei}')

In [ ]:
results_Ni = []
results_NiAl = []
results_Ni3Al = []

with open('data/01_lattice-constant_Ni.dat') as f:
    for line in f:
        result = [element.strip() for element in line.split(',')]
        results_Ni.append(result)
        
with open('data/01_lattice-constant_NiAl.dat') as f:
    for line in f:
        result = [element.strip() for element in line.split(',')]
        results_NiAl.append(result)
        
with open('data/01_lattice-constant_Ni3Al.dat') as f:
    for line in f:
        result = [element.strip() for element in line.split(',')]
        results_Ni3Al.append(result)

In [ ]:
all_files = []
alloy_files = []

cohesiveenergy_NiAl = []
cohesiveenergy_Ni3Al = []

latticeconstant_Ni = []
latticeconstant_NiAl = []
latticeconstant_Ni3Al = []

for i in range(len(results_Ni)):
    potentialname = results_Ni[i][0]
    latticeconstant = results_Ni[i][2]
    
    all_files.append(potentialname)  
    latticeconstant_Ni.append(latticeconstant)
    
for i in range(len(results_NiAl)):
    potentialname = results_NiAl[i][0]
    cohesiveenergy = results_NiAl[i][1]
    latticeconstant = results_NiAl[i][2]
    
    alloy_files.append(potentialname)
    cohesiveenergy_NiAl.append(cohesiveenergy)
    latticeconstant_NiAl.append(latticeconstant)
    
for i in range(len(results_Ni3Al)):
    potentialname = results_Ni3Al[i][0]
    cohesiveenergy = results_Ni3Al[i][1]
    latticeconstant = results_Ni3Al[i][2]
    
    alloy_files.append(potentialname)
    cohesiveenergy_Ni3Al.append(cohesiveenergy)
    latticeconstant_Ni3Al.append(latticeconstant)

In [ ]:
for file, lc in zip(all_files, latticeconstant_Ni):
    from lammps import PyLammps
    lmp = PyLammps()

    potentialpath = "potentials/"+file
    
    vacancyformation("Ni", "Ni", "fcc", lc, "V_Ni", potentialpath)

    lmp.print(f'"{file}'+', ${E_V_Ni}" append data/03_point-defects_Ni.dat screen no')

In [ ]:
compound = ["NiAl", "Ni3Al"]
whichelements = ["Ni Al", "Ni Al"]
crystalstructure = ["bcc", "fcc"]
cohesiveenergies = [cohesiveenergy_NiAl, cohesiveenergy_Ni3Al]
latticeconstants = [latticeconstant_NiAl, latticeconstant_Ni3Al]

for comp, element, crystal, cohesiveenergy, latticeconstant in zip (compound, whichelements, crystalstructure, cohesiveenergies, latticeconstants):
    for file, ecoh, lc in zip(alloy_files, cohesiveenergy, latticeconstant):
        
        from lammps import PyLammps
        lmp = PyLammps()
        
        potentialpath = "potentials/"+file
        outputfile = "data/03_point-defects_"+comp+".dat"
                
        for pd in ["V_Ni", "Al_Ni", "V_Al", "Ni_Al"]:
            vacancyformation(comp, element, crystal, lc, pd, potentialpath)
            
        if comp == "NiAl":
            lmp.variable('E_eff equal ${E_V_Ni}+'+f'{ecoh}'+'+(${E_Ni_Al}-${E_Al_Ni})/4')

        elif comp == "Ni3Al":
            lmp.variable('E_eff equal ${E_V_Ni}+'+f'{ecoh}'+'+(${E_Ni_Al}-${E_Al_Ni})/8')
        
        lmp.print(f'"{file}'+', ${E_V_Ni}, ${E_Al_Ni}, ${E_V_Al}, ${E_Ni_Al}, ${E_eff}" append '+f'{outputfile} screen no')

🏁**FAST RUN DONE!**🏁